In [ ]:
##import libraries
import warnings
import pandas as pd
import numpy as np 
import seaborn as sns
import scipy.stats as stats
import csv
import scipy
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression


In [ ]:
#read in step 1 results for never smokers
ancestry = 'EUR'
pgs_file = 'PGS002037'
phecode_df = pd.read_csv(('./path/to/step1/results').format(pgs_file, ancestry), sep = ',')
phecode_df.head()

In [ ]:
#restrict to rows withpgs
z_df = phecode_df.loc[phecode_df['Unnamed: 0'].isin(['PRS_Z_eur'])]
z_df

In [ ]:
#fix column name
rename_df = z_df.rename(columns={'P>|z|': 'p_value_z'})
rename_df

In [ ]:
#get phenotype names
phecode_defns_df = pd.read_csv("./path/to/phecode/definitions", sep=',')
phecode_defns_df['phecode'] = phecode_defns_df['phecode']
phecode_defns_df.head()

In [ ]:
phecode_defns_df.category.value_counts()

In [ ]:
rename_df.phecode.dtype

In [ ]:
#merge names with results
df = rename_df.merge(phecode_defns_df[['phecode','phenotype', 'category']], on='phecode', how='left')
df.head()

In [ ]:
df.sort_values('p_value_z').head(50)

In [ ]:
#minuslogp column
df['minuslogp'] = -np.log10(df.p_value_z); df = df.sort_values(['category'])
df.replace(np.inf, 0, inplace=True)
df = df.dropna(subset=['minuslogp'])
df.reset_index(inplace=True, drop=True); df['i'] = df.index
df["minuslogp"] = pd.to_numeric(df["minuslogp"])
df

In [ ]:
df = df.fillna("other")

In [ ]:
df.to_csv('./output/for/meta/analysis'.format(pgs_file, ancestry))

In [ ]:
df.category.nunique()

In [ ]:
0.05/1847

# Phewas Z plot

In [ ]:
# Generate PHEWAS plot with Z
phewasplot = sns.relplot(data=df, x='i', y='z', aspect=3, 
                   hue='category', palette=sns.color_palette('bright', n_colors=18), legend='full')
for line in range(0, df.shape[0]):
    plt.text(
        df.i[line],
        df.z[line],
        df.phenotype[line] if df.z[line] >4.15  or  df.z[line] < -4.15 else "",
        verticalalignment= "top" if line %2 else "top",
        horizontalalignment="left" if line %-5 else "left",
        rotation =20,
        wrap=3,
        size='small', color='black', fontsize=14, weight='normal')
CATEGORY_phewas=df.groupby('category')['i'].median()
phewasplot.ax.axhline(-4, linestyle='--', linewidth=1, color="red")
phewasplot.ax.axhline(4, linestyle='--', linewidth=1, color="red")
for ax in phewasplot.axes.flat: ax.spines['bottom'].set_position('zero')
phewasplot.ax.set_xlabel('category')
phewasplot.ax.set_xticks(CATEGORY_phewas)
phewasplot.ax.set_xticklabels(CATEGORY_phewas.index, rotation = 30, horizontalalignment="right")
phewasplot.fig.suptitle('PRS_{}_PHEWAS_{}'.format(pgs_file,ancestry), x=0.1, weight = 'bold');